In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from PIL import Image
import torch
import torchvision.transforms.functional as TF

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
  print(torch.cuda.current_device())
  print(torch.cuda.device(0))
  print(torch.cuda.device_count())
  print(torch.cuda.get_device_name(0))
else:
  print("No NVIDIA driver found. Using CPU")

0
1
NVIDIA GeForce MX350


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class ColorizationNet(nn.Module):
    def __init__(self):
        super(ColorizationNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=4, dilation=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=4, dilation=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=4, dilation=2)
        self.conv4 = nn.Conv2d(128, 3, kernel_size=5, stride=1, padding=4, dilation=2)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.relu(self.conv3(x))
        x = torch.sigmoid(self.conv4(x))
        return x

In [5]:
model = ColorizationNet().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def rgb_to_gray(img):
    return img.mean(dim=1, keepdim=True)

In [8]:
EPOCHS = 5
training_losses = []
for epoch in range(EPOCHS):
    for i, (images, _) in enumerate(train_loader):
        grayscale_images = rgb_to_gray(images).to(device)
        images = images.to(device)

        outputs = model(grayscale_images)
        loss = criterion(outputs, images)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        training_losses.append(loss.item())

        if i % 300 == 0:
            print(f"Epoch [{epoch+1}/{EPOCHS}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

# حفظ النموذج بعد الانتهاء من التدريب
torch.save(model.state_dict(), 'final_model.pth')  # حفظ النموذج بعد التدريب بالكامل

print("Finished Training")


Epoch [1/5], Step [1/782], Loss: 0.0047
Epoch [1/5], Step [301/782], Loss: 0.0056
Epoch [1/5], Step [601/782], Loss: 0.0046
Epoch [2/5], Step [1/782], Loss: 0.0048
Epoch [2/5], Step [301/782], Loss: 0.0047
Epoch [2/5], Step [601/782], Loss: 0.0047
Epoch [3/5], Step [1/782], Loss: 0.0077
Epoch [3/5], Step [301/782], Loss: 0.0056
Epoch [3/5], Step [601/782], Loss: 0.0053
Epoch [4/5], Step [1/782], Loss: 0.0043
Epoch [4/5], Step [301/782], Loss: 0.0057
Epoch [4/5], Step [601/782], Loss: 0.0058
Epoch [5/5], Step [1/782], Loss: 0.0044
Epoch [5/5], Step [301/782], Loss: 0.0059
Epoch [5/5], Step [601/782], Loss: 0.0047
Finished Training


In [9]:
import streamlit as st
import torch
import torchvision.transforms as transforms
from PIL import Image
from moviepy.editor import VideoFileClip
import numpy as np
import matplotlib.pyplot as plt
from torch import nn, optim
import torchvision

# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained model
class ColorizationNet(nn.Module):
    def __init__(self):
        super(ColorizationNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=4, dilation=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=4, dilation=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=4, dilation=2)
        self.conv4 = nn.Conv2d(128, 3, kernel_size=5, stride=1, padding=4, dilation=2)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.relu(self.conv3(x))
        x = torch.sigmoid(self.conv4(x))
        return x

model = ColorizationNet().to(device)
model.load_state_dict(torch.load('final_model.pth'))  # Assuming you've already trained and saved your model

# Define transformation
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Streamlit UI
st.title("AI Image and Video Colorizer")

# Image Upload
uploaded_image = st.file_uploader("Upload an Image", type=["jpg", "jpeg", "png"])
if uploaded_image:
    img = Image.open(uploaded_image)
    st.image(img, caption="Uploaded Image", use_column_width=True)

    # Transform and colorize the image
    img_tensor = transform(img).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        colorized_tensor = model(img_tensor)
    
    colorized_img = transforms.ToPILImage()(colorized_tensor.squeeze(0).cpu())
    st.image(colorized_img, caption="Colorized Image", use_column_width=True)

# Video Upload
uploaded_video = st.file_uploader("Upload a Video", type=["mp4", "mov", "avi"])
if uploaded_video:
    clip = VideoFileClip(uploaded_video)
    st.video(uploaded_video)

    # Colorize video frames
    colorized_frames = []
    for frame in clip.iter_frames():
        frame_gray_tensor = transform(Image.fromarray(frame)).unsqueeze(0).to(device)
        with torch.no_grad():
            frame_colorized = model(frame_gray_tensor)
        colorized_frames.append(frame_colorized.cpu())
    
    # Save colorized video
    output_video_path = "colorized_output.mp4"
    colorized_clip = VideoFileClip(uploaded_video)
    colorized_clip = colorized_clip.set_fps(clip.fps)

    def colorize_frame(t):
        index = min(int(t * clip.fps), len(colorized_frames) - 1)
        frame_colorized = colorized_frames[index].squeeze(0).permute(1, 2, 0).numpy()
        return (255 * frame_colorized).astype('uint8')

    colorized_clip = colorized_clip.fl(lambda gf, t: colorize_frame(t), apply_to=['mask', 'audio'])
    colorized_clip.write_videofile(output_video_path, codec='libx264')

    # Provide the colorized video download link
    st.download_button("Download Colorized Video", data=open(output_video_path, "rb"), file_name="colorized_video.mp4")


2024-12-27 17:34:52.247 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-27 17:34:52.666 
  command:

    streamlit run c:\Users\mohammed\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-27 17:34:52.667 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-27 17:34:52.668 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-27 17:34:52.668 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-27 17:34:52.668 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-27 17:34:52.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-27 17:34:52.670 Thread 'MainThread':